# Домашнее задание к лекции Улучшение качества модели

Взять boston house-prices datase (sklearn.datasets.load_boston). Возьмите 7 любых регрессоров (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество).

Импортируем датасет для работы и методы улучшения качества модели, которые будут работать с гиперпараметрами наших регрессоров.

In [2]:
from sklearn.datasets import load_boston
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn import metrics
import matplotlib.pyplot as plt

In [3]:
boston = load_boston()

X = boston.data
y = boston.target

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 21, test_size = 0.25)

## RandomizedSearchCV

Импортируем все регрессоры, которые интересно посмотреть в данной задаче.

In [8]:
from sklearn.linear_model import Ridge, BayesianRidge, TweedieRegressor, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor

Ниже метод для игнорирования предупреждений в процессе работы

In [9]:
import warnings
warnings.filterwarnings("ignore")

Поместим все регрессоры с некоторыми гиперпараметрами и некоторыми значениями в список models. Далее будем пробегаться по нему и прогонять через оптимизатор, сначала через RandomizedSearchCV, а затем через GridSearchCV. Далее выводить метрика качества.

In [10]:
models = [
    {'name':'R',"model": Ridge(), 'params':{'solver':['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}},
    {'name':'BR',"model": BayesianRidge(), 'params':{'alpha_1':[1e-6, 1e-7, 1e-5], 'lambda_1':[1e-6, 1e-7, 1e-5]}},
    {'name':'TwR',"model": TweedieRegressor(), 'params':{'power':[0, 1, 2, 3], 'alpha':[0, 0.25, 0.5, 0.75, 1]}},
    {'name':'SGD',"model": SGDRegressor(), 'params':{'penalty':['l2', 'l1', 'elasticnet'], 'learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'], 'power_t':[0.1, 0.2, 0.5, 0.7]}},
    {'name':'RFor',"model": RandomForestRegressor(), 'params':{'n_estimators': [50, 75, 100, 125], 'max_features': ['auto', 'sqrt', 'log2']}},
    {'name':'DTr',"model": DecisionTreeRegressor(), 'params':{'criterion':['mse', 'mae', 'poisson', 'friedman_mse'], 'splitter': ['best', 'random']}},
    {'name':'SVR',"model": SVR(), 'params':{'degree':[1, 2, 3, 4]}},
    {'name':'GBooR',"model": GradientBoostingRegressor(), 'params':{'loss':['ls', 'lad', 'huber', 'quantile'], 'max_depth':[3,5,7,9,11], 'n_estimators':[75, 100, 125], 'criterion': ['friedman_mse', 'mse', 'mae']}},
    {'name':'KR',"model": KernelRidge(), 'params':{'degree':[1, 2, 3, 4, 5]}},
    {'name':'KNeigh',"model": KNeighborsRegressor(), 'params':{'n_neighbors':[3, 4, 5, 7, 9], 'weights':['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}}
]

res = []
for v in models:
    res.append((v['name'], RandomizedSearchCV(v['model'], v['params'], cv=10).fit(X_train, y_train)))

In [11]:
for r in res:
    print(r[0], r[1].best_score_, r[1].best_params_)

R 0.7081684796000693 {'solver': 'svd'}
BR 0.6976310910353944 {'lambda_1': 1e-07, 'alpha_1': 1e-05}
TwR 0.7592429678495294 {'power': 1, 'alpha': 0.5}
SGD -5.547116447238407e+21 {'power_t': 0.7, 'penalty': 'l1', 'learning_rate': 'invscaling'}
RFor 0.856833610794857 {'n_estimators': 100, 'max_features': 'log2'}
DTr 0.7065813251781716 {'splitter': 'best', 'criterion': 'mae'}
SVR 0.20234179199604027 {'degree': 1}
GBooR 0.8395594278509391 {'n_estimators': 125, 'max_depth': 9, 'loss': 'lad', 'criterion': 'friedman_mse'}
KR 0.6711343480075428 {'degree': 1}
KNeigh 0.5470430341221104 {'weights': 'distance', 'n_neighbors': 9, 'algorithm': 'ball_tree'}


При оптимизаторе RandomizedSearchCV лучше всего справился регрессор RandomForestRegressor - 0.857, затем GradientBoostingRegressor - 0.840 и т.д.

## GridSearchCV

In [12]:
res_grid = []
for w in models:
    res_grid.append((w['name'], GridSearchCV(w['model'], w['params'], cv=10).fit(X_train, y_train)))

In [13]:
for r in res_grid:
    print(r[0], r[1].best_score_, r[1].best_params_)

R 0.7081684796000693 {'solver': 'svd'}
BR 0.6976310910353944 {'alpha_1': 1e-05, 'lambda_1': 1e-07}
TwR 0.7663448806170917 {'alpha': 0, 'power': 1}
SGD -1.1101914595001407e+21 {'learning_rate': 'invscaling', 'penalty': 'l2', 'power_t': 0.7}
RFor 0.854803887374295 {'max_features': 'sqrt', 'n_estimators': 50}
DTr 0.7416904712660894 {'criterion': 'mae', 'splitter': 'random'}
SVR 0.20234179199604027 {'degree': 1}
GBooR 0.8583827723593952 {'criterion': 'mse', 'loss': 'huber', 'max_depth': 3, 'n_estimators': 100}
KR 0.6711343480075428 {'degree': 1}
KNeigh 0.5470430341221104 {'algorithm': 'auto', 'n_neighbors': 9, 'weights': 'distance'}


При оптимизаторе GridSearchCV лучше всего справился регрессор GradientBoostingRegressor - 0.858, затем RandomForestRegressor - 0.855 и т.д. Надо отметить, что время потраченное на оптимизацию значительно выше, чем в оптимизаторе выше, это и объясняется его концепцией работы.

Если сравнивать качества между этими двумя оптимизаторами, то GridSearchCV немного выше на тысячные доли, если важно достичь результатов тысячных долей и данных не слишком много, и есть мощности для работы, то GridSearchCV простой и действенный выбор оптимизации гиперпараметров, а если важнее время на больших объемах данных, то тогда RandomizedSearchCV.